In [1]:
!pip install --quiet google-cloud-aiplatform google-cloud-storage vertexai pandas numpy langchain tiktoken python-dateutil sentence-transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 45.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.13.0 requires google-cloud-aiplatform[agent-engines]<2.0.0,>=1.95.1, but you have google-cloud-aiplatform 1.71.1 which is incompatible.


In [2]:
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=NzT1dvq5f9FJsZjpVmDNTilmYcnwoY&prompt=consent&token_usage=remote&access_type=offline&code_challenge=hvaY2GMdjxz8xwWbfSf7AIE4qV-nBeMSWMLRNby1AAQ&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJihXIDjvKzypPLtR2vB9vE1H6SWWxnt3n0EkAwxthsILaK3Tl35FCJcFtz7_03hLA

You are now logged in as [jonahprashanth@gmail.com].
Your current proje

In [3]:
!gcloud config set project i-monolith-468706-i9

Updated property [core/project].


In [4]:
!gcloud auth list

     Credentialed Accounts
ACTIVE  ACCOUNT
*       jonahprashanth@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
!gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = jonahprashanth@gmail.com
project = i-monolith-468706-i9

Your active configuration is: [default]


In [6]:
!gcloud iam service-accounts create vertex-sa \
  --display-name "Vertex AI Service Account" \
  --project i-monolith-468706-i9

ERROR: (gcloud.iam.service-accounts.create) Resource in projects [i-monolith-468706-i9] is the subject of a conflict: Service account vertex-sa already exists within project projects/i-monolith-468706-i9.
- '@type': type.googleapis.com/google.rpc.ResourceInfo
  resourceName: projects/i-monolith-468706-i9/serviceAccounts/vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com


In [7]:
!gcloud projects add-iam-policy-binding i-monolith-468706-i9 \
  --member="serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com" \
  --role="roles/aiplatform.user"

!gcloud projects add-iam-policy-binding i-monolith-468706-i9 \
  --member="serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com" \
  --role="roles/storage.objectAdmin"


Updated IAM policy for project [i-monolith-468706-i9].
bindings:
- members:
  - serviceAccount:sample-test@i-monolith-468706-i9.iam.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-393707886745@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:sample-test@i-monolith-468706-i9.iam.gserviceaccount.com
  - serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com
  - user:jonahprashanth@gmail.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-393707886745@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:393707886745@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-393707886745@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-393707886745@compute-system.iam.gse

In [8]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable storage.googleapis.com

Operation "operations/acat.p2-393707886745-d5446926-2edb-4b49-8d00-25ced7a86204" finished successfully.


In [9]:
!gcloud iam service-accounts keys create key.json \
  --iam-account=vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com


created key [058d8da7223d9970a6d3743ed590e919d2185fba] of type [json] as [key.json] for [vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com]


In [10]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"


In [11]:
PROJECT_ID = "i-monolith-468706-i9"   # your GCP project
BUCKET_NAME = "bucket_jonah"          # change if you want a new bucket
LOCATION = "us-central1"              # or us-east1, asia-south1, etc.

# --- 2. Create bucket (only if not already exists) ---
!gcloud storage buckets create gs://$BUCKET_NAME \
    --project=$PROJECT_ID \
    --location=$LOCATION \
    --default-storage-class=STANDARD \
    --uniform-bucket-level-access \
    --quiet || echo "Bucket may already exist"

Creating gs://bucket_jonah/...


In [12]:
import os
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime
from typing import List, Dict, Any, Optional
import ast
import warnings
import time
from sklearn.metrics.pairwise import cosine_similarity
from google.cloud import storage
import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from langchain.text_splitter import RecursiveCharacterTextSplitter

warnings.filterwarnings('ignore')

# Configuration
PROJECT_ID = "i-monolith-468706-i9"
LOCATION = "us-central1"
BUCKET_NAME = "bucket_jonah"  # Update with your bucket name

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

class MedicalDataProcessor:
    """Handles CSV data cleaning and preprocessing"""

    def __init__(self):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,  # 1.5KB chunks
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def clean_csv_data(self, csv_path: str) -> pd.DataFrame:
        """Clean and preprocess the EHR CSV data"""
        print("Loading CSV data...")
        df = pd.read_csv(csv_path)

        print(f"Original data shape: {df.shape}")

        # Basic cleaning
        df = df.dropna(subset=['TEXT'])  # Remove rows without medical text
        df = df.drop_duplicates(subset=['TEXT'])  # Remove duplicate notes

        # Clean text content
        df['TEXT'] = df['TEXT'].apply(self._clean_medical_text)

        # Parse JSON fields safely
        df['diagnoses_parsed'] = df['diagnoses'].apply(self._safe_parse_json)
        df['procedures_parsed'] = df['procedures'].apply(self._safe_parse_json)
        df['cpt_codes_parsed'] = df['cpt_codes'].apply(self._safe_parse_json)

        # Standardize dates
        df['CHARTDATE'] = pd.to_datetime(df['CHARTDATE'], errors='coerce')

        # Remove rows with no medical codes
        df = df[
            (df['diagnoses_parsed'].notna()) |
            (df['procedures_parsed'].notna()) |
            (df['cpt_codes_parsed'].notna())
        ]

        print(f"Cleaned data shape: {df.shape}")
        return df

    def _clean_medical_text(self, text: str) -> str:
        """Clean medical text content"""
        if pd.isna(text) or not isinstance(text, str):
            return ""

        # Remove common PHI patterns (basic anonymization)
        text = re.sub(r'\[\*\*.*?\*\*\]', '[REDACTED]', text)  # Remove bracketed PHI

        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        # Remove control characters
        text = ''.join(char for char in text if ord(char) >= 32 or char == '\n')

        return text

    def _safe_parse_json(self, json_str: str) -> Optional[List[Dict]]:
        """Safely parse JSON strings from CSV"""
        if pd.isna(json_str) or json_str == '':
            return None

        try:
            # Handle string representation of list
            if isinstance(json_str, str):
                # Try ast.literal_eval first
                parsed = ast.literal_eval(json_str)
                return parsed if isinstance(parsed, list) else None
        except (ValueError, SyntaxError):
            try:
                # Try json.loads as backup
                return json.loads(json_str)
            except json.JSONDecodeError:
                return None
        return None

    def _extract_codes_as_strings(self, codes_list: Optional[List[Dict]], code_key: str) -> List[str]:
        """Extract codes from parsed JSON and convert to strings"""
        if not codes_list:
            return []

        result = []
        for item in codes_list:
            if isinstance(item, dict) and code_key in item:
                code = item[code_key]
                if code is not None and str(code).strip():
                    result.append(str(code).strip())
        return result

    def _extract_titles_as_strings(self, codes_list: Optional[List[Dict]], title_key: str) -> List[str]:
        """Extract titles from parsed JSON and convert to strings"""
        if not codes_list:
            return []

        result = []
        for item in codes_list:
            if isinstance(item, dict) and title_key in item:
                title = item[title_key]
                if title is not None and str(title).strip():
                    result.append(str(title).strip())
        return result

    def chunk_documents(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        """Split long medical texts into chunks"""
        chunks = []

        for idx, row in df.iterrows():
            text = row['TEXT']
            if not text or len(text.strip()) < 50:
                continue

            # Split text into chunks
            text_chunks = self.text_splitter.split_text(text)

            for chunk_idx, chunk in enumerate(text_chunks):
                chunk_data = {
                    'content': chunk,
                    'original_row_id': int(row['row_id']),
                    'subject_id': int(row['SUBJECT_ID']),
                    'admission_id': int(row['HADM_ID']),
                    'chart_date': row['CHARTDATE'].isoformat() if pd.notna(row['CHARTDATE']) else None,
                    'category': str(row['CATEGORY']),
                    'description': str(row['DESCRIPTION']),
                    'chunk_id': chunk_idx,
                    'total_chunks': len(text_chunks),

                    # Medical codes
                    'diagnoses': row['diagnoses_parsed'] or [],
                    'procedures': row['procedures_parsed'] or [],
                    'cpt_codes': row['cpt_codes_parsed'] or [],

                    # Extract code strings properly
                    'diagnosis_codes': self._extract_codes_as_strings(row['diagnoses_parsed'], 'ICD9_CODE'),
                    'diagnosis_titles': self._extract_titles_as_strings(row['diagnoses_parsed'], 'LONG_TITLE'),
                    'procedure_codes': self._extract_codes_as_strings(row['procedures_parsed'], 'ICD9_CODE'),
                    'procedure_titles': self._extract_titles_as_strings(row['procedures_parsed'], 'LONG_TITLE'),
                    'cpt_code_values': self._extract_codes_as_strings(row['cpt_codes_parsed'], 'CPT_CD'),
                    'cpt_descriptions': self._extract_titles_as_strings(row['cpt_codes_parsed'], 'DESCRIPTION')
                }
                chunks.append(chunk_data)

        print(f"Created {len(chunks)} text chunks")
        return chunks

class VertexAIMedicalRAG:
    """Medical RAG system using Google Cloud Storage and Vertex AI"""

    def __init__(self, project_id: str, location: str, bucket_name: str):
        self.project_id = project_id
        self.location = location
        self.bucket_name = bucket_name

        # Initialize Vertex AI
        vertexai.init(project=project_id, location=location)

        # Initialize models
        self.embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
        self.generative_model = GenerativeModel("gemini-2.5-flash")

        # Initialize GCS client
        self.storage_client = storage.Client(project=project_id)
        self.bucket = self.storage_client.bucket(bucket_name)

        # In-memory storage for embeddings (loaded from GCS)
        self.medical_records = []

        print(f"✅ Initialized Vertex AI Medical RAG")
        print(f"   Project: {project_id}")
        print(f"   Location: {location}")
        print(f"   Bucket: {bucket_name}")

    def generate_embeddings(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Generate embeddings using Vertex AI in batches"""
        all_embeddings = []

        print(f"Generating embeddings for {len(texts)} texts in batches of {batch_size}...")

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                embeddings = self.embedding_model.get_embeddings(batch)
                batch_embeddings = [e.values for e in embeddings]
                all_embeddings.extend(batch_embeddings)

                print(f"  Processed batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")
                time.sleep(0.5)  # Rate limiting

            except Exception as e:
                print(f"❌ Error in batch {i//batch_size + 1}: {e}")
                # Add empty embeddings for failed batch
                empty_embeddings = [[0.0] * 768 for _ in batch]  # text-embedding-004 has 768 dimensions
                all_embeddings.extend(empty_embeddings)

        return all_embeddings

    def create_embeddings_and_store(self, chunks: List[Dict[str, Any]]):
        """Create embeddings for chunks and store in GCS"""
        print("=== EMBEDDING CREATION PHASE ===")

        # Extract texts for embedding
        texts = [chunk['content'] for chunk in chunks]

        # Generate embeddings
        embeddings = self.generate_embeddings(texts)

        # Combine chunks with embeddings
        medical_records = []
        for chunk, embedding in zip(chunks, embeddings):
            record = {
                **chunk,  # Include all chunk data
                'embedding': embedding,
                'embedding_id': f"{chunk['original_row_id']}_{chunk['chunk_id']}"
            }
            medical_records.append(record)

        # Save to local JSONL first
        os.makedirs("embeddings", exist_ok=True)
        local_path = "embeddings/medical_embeddings.jsonl"

        with open(local_path, "w") as f:
            for record in medical_records:
                f.write(json.dumps(record, default=str) + "\n")

        print(f"✅ Saved {len(medical_records)} records to {local_path}")

        # Upload to GCS
        gcs_path = "embeddings/medical_embeddings.jsonl"
        blob = self.bucket.blob(gcs_path)
        blob.upload_from_filename(local_path)

        print(f"⬆️ Uploaded to gs://{self.bucket_name}/{gcs_path}")

        # Store in memory for immediate use
        self.medical_records = medical_records
        print(f"✅ Loaded {len(self.medical_records)} records into memory")

    def load_embeddings_from_gcs(self):
        """Load embeddings from GCS into memory"""
        try:
            gcs_path = "embeddings/medical_embeddings.jsonl"
            blob = self.bucket.blob(gcs_path)

            # Download to local file
            local_path = "/tmp/medical_embeddings.jsonl"
            blob.download_to_filename(local_path)

            # Load into memory
            self.medical_records = []
            with open(local_path, "r") as f:
                for line in f:
                    record = json.loads(line.strip())
                    self.medical_records.append(record)

            print(f"✅ Loaded {len(self.medical_records)} records from GCS")

        except Exception as e:
            print(f"❌ Error loading from GCS: {e}")
            print("Please run embedding creation first.")

    def search_similar_records(self, query: str, top_k: int = 10) -> List[Dict[str, Any]]:
        """Search for similar medical records using cosine similarity"""
        if not self.medical_records:
            print("❌ No medical records loaded. Please load embeddings first.")
            return []

        print(f"🔍 Searching for: '{query}'")

        # Generate query embedding
        query_embeddings = self.embedding_model.get_embeddings([query])
        query_vector = np.array([query_embeddings[0].values])

        # Get all record embeddings
        record_embeddings = np.array([record['embedding'] for record in self.medical_records])

        # Calculate similarities
        similarities = cosine_similarity(query_vector, record_embeddings)[0]

        # Get top-k similar records
        top_indices = np.argsort(similarities)[::-1][:top_k]

        similar_records = []
        for idx in top_indices:
            record = self.medical_records[idx].copy()
            record['similarity_score'] = float(similarities[idx])
            similar_records.append(record)

        return similar_records

    def extract_medical_codes_from_results(self, similar_records: List[Dict[str, Any]]) -> Dict[str, Dict]:
        """Extract and aggregate medical codes from search results"""
        medical_codes = {
            'diagnoses': {},
            'procedures': {},
            'cpt_codes': {}
        }

        for record in similar_records:
            # Process diagnoses
            for diag in record.get('diagnoses', []):
                code = diag.get('ICD9_CODE')
                if code:
                    if code not in medical_codes['diagnoses']:
                        medical_codes['diagnoses'][code] = {
                            'short_title': diag.get('SHORT_TITLE', ''),
                            'long_title': diag.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['diagnoses'][code]['frequency'] += 1
                    medical_codes['diagnoses'][code]['max_similarity'] = max(
                        medical_codes['diagnoses'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process procedures
            for proc in record.get('procedures', []):
                code = str(proc.get('ICD9_CODE'))
                if code and code != 'None':
                    if code not in medical_codes['procedures']:
                        medical_codes['procedures'][code] = {
                            'short_title': proc.get('SHORT_TITLE', ''),
                            'long_title': proc.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['procedures'][code]['frequency'] += 1
                    medical_codes['procedures'][code]['max_similarity'] = max(
                        medical_codes['procedures'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process CPT codes
            for cpt in record.get('cpt_codes', []):
                code = cpt.get('CPT_CD')
                if code:
                    if code not in medical_codes['cpt_codes']:
                        medical_codes['cpt_codes'][code] = {
                            'description': cpt.get('DESCRIPTION', ''),
                            'section': cpt.get('SECTIONHEADER', ''),
                            'subsection': cpt.get('SUBSECTIONHEADER', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['cpt_codes'][code]['frequency'] += 1
                    medical_codes['cpt_codes'][code]['max_similarity'] = max(
                        medical_codes['cpt_codes'][code]['max_similarity'],
                        record['similarity_score']
                    )

        return medical_codes

    def generate_medical_response(self, query: str, similar_records: List[Dict[str, Any]], medical_codes: Dict[str, Dict]) -> str:
        """Generate explanation using Vertex AI Gemini"""

        # Prepare context from top records
        context = "\n".join([
            f"Record {i+1}: {record['content'][:300]}..."
            for i, record in enumerate(similar_records[:3])
        ])

        # Format medical codes
        diagnoses_text = ""
        if medical_codes['diagnoses']:
            diagnoses_text = "DIAGNOSIS CODES FOUND:\n"
            for code, info in sorted(medical_codes['diagnoses'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                diagnoses_text += f"- {code}: {info['long_title']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        procedures_text = ""
        if medical_codes['procedures']:
            procedures_text = "PROCEDURE CODES FOUND:\n"
            for code, info in sorted(medical_codes['procedures'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                procedures_text += f"- {code}: {info['long_title']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        cpt_text = ""
        if medical_codes['cpt_codes']:
            cpt_text = "CPT CODES FOUND:\n"
            for code, info in sorted(medical_codes['cpt_codes'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                cpt_text += f"- {code}: {info['description']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        prompt = f"""
You are a medical coding assistant. Based on the following medical records context and the user's query about diseases/symptoms, provide relevant medical codes and explanations.

CONTEXT FROM MEDICAL RECORDS:
{context}

{diagnoses_text}

{procedures_text}

{cpt_text}

USER QUERY: {query}

Please provide a comprehensive response that:
1. Explains what medical codes are most relevant to the query
2. Provides context on why these codes match the query
3. Lists the specific ICD9 diagnosis codes, procedure codes, and CPT codes
4. Explains what each code represents in simple terms
5. Notes the similarity scores to show relevance

Keep your response concise and well-formatted.
"""

        try:
            response = self.generative_model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating explanation: {e}. However, medical codes were found successfully."

    def query(self, user_query: str, top_k: int = 10) -> Dict[str, Any]:
        """Main query interface"""
        print(f"Processing query: {user_query}")

        # Search for similar records
        similar_records = self.search_similar_records(user_query, top_k)

        if not similar_records:
            return {
                'query': user_query,
                'explanation': "No similar medical records found.",
                'medical_codes': {'diagnoses': {}, 'procedures': {}, 'cpt_codes': {}},
                'retrieved_records_count': 0,
                'sample_records': []
            }

        # Extract medical codes
        medical_codes = self.extract_medical_codes_from_results(similar_records)

        # Generate explanation
        explanation = self.generate_medical_response(user_query, similar_records, medical_codes)

        # Prepare sample records
        sample_records = []
        for record in similar_records[:3]:
            sample_records.append({
                'content': record['content'][:200] + "..." if len(record['content']) > 200 else record['content'],
                'category': record['category'],
                'description': record['description'],
                'similarity_score': record['similarity_score'],
                'row_id': record['original_row_id']
            })

        return {
            'query': user_query,
            'explanation': explanation,
            'medical_codes': medical_codes,
            'retrieved_records_count': len(similar_records),
            'sample_records': sample_records
        }

In [16]:

def main():
    """Main execution function"""

    # Configuration
    CSV_PATH = "/content/ehr_records.csv"  # Update with your CSV path

    # Initialize components
    processor = MedicalDataProcessor()
    rag_system = VertexAIMedicalRAG(PROJECT_ID, LOCATION, BUCKET_NAME)

    # Setup mode
    setup_mode = input("Do you want to setup the database? (y/n): ").lower() == 'y'

    if setup_mode:
        print("=== DATA PROCESSING PHASE ===")
        df = processor.clean_csv_data(CSV_PATH)
        chunks = processor.chunk_documents(df)

        print("=== EMBEDDING & STORAGE PHASE ===")
        rag_system.create_embeddings_and_store(chunks)
        print("Setup complete!")
    else:
        # Load existing embeddings
        print("=== LOADING EMBEDDINGS FROM GCS ===")
        rag_system.load_embeddings_from_gcs()

    # Interactive query mode
    print("\n=== MEDICAL RAG SYSTEM READY ===")
    print("Enter medical queries to find relevant codes. Type 'quit' to exit.\n")

    # Sample queries
    sample_queries = [
        "chest pain and shortness of breath",
        "cardiac arrest and heart failure",
        "difficulty breathing",
        "hypotension and shock",
        "mechanical ventilation"
    ]

    print("Sample queries you can try:")
    for i, q in enumerate(sample_queries, 1):
        print(f"  {i}. {q}")
    print()

    while True:
        query = input("Enter your medical query (or 'quit' to exit): ").strip()
        if query.lower() in ['quit', 'exit', 'q']:
            break

        if not query:
            continue

        try:
            results = rag_system.query(query)

            print("\n" + "="*60)
            print(f"QUERY: {results['query']}")
            print("="*60)

            print("🏥 MEDICAL CODES FOUND:")

            # Display diagnoses
            if results['medical_codes']['diagnoses']:
                print("\n📋 DIAGNOSIS CODES (ICD9):")
                for code, info in results['medical_codes']['diagnoses'].items():
                    print(f"  • {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            # Display procedures
            if results['medical_codes']['procedures']:
                print("\n🔬 PROCEDURE CODES (ICD9):")
                for code, info in results['medical_codes']['procedures'].items():
                    print(f"  • {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            # Display CPT codes
            if results['medical_codes']['cpt_codes']:
                print("\n💊 CPT CODES:")
                for code, info in results['medical_codes']['cpt_codes'].items():
                    print(f"  • {code}: {info['description']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            if not any([
                results['medical_codes']['diagnoses'],
                results['medical_codes']['procedures'],
                results['medical_codes']['cpt_codes']
            ]):
                print("  No specific medical codes found for this query.")

            print(f"\n📄 EXPLANATION:")
            print(results['explanation'])

            print(f"\n📊 Retrieved {results['retrieved_records_count']} relevant medical records")
            print("="*60 + "\n")

        except Exception as e:
            print(f"❌ Error processing query: {e}")
            print("Please try again with a different query.\n")

if __name__ == "__main__":
    main()

✅ Initialized Vertex AI Medical RAG
   Project: i-monolith-468706-i9
   Location: us-central1
   Bucket: bucket_jonah
Do you want to setup the database? (y/n): y
=== DATA PROCESSING PHASE ===
Loading CSV data...
Original data shape: (1976, 14)
Cleaned data shape: (1945, 17)
Created 3710 text chunks
=== EMBEDDING & STORAGE PHASE ===
=== EMBEDDING CREATION PHASE ===
Generating embeddings for 3710 texts in batches of 32...
  Processed batch 1/116
  Processed batch 2/116
  Processed batch 3/116
  Processed batch 4/116
  Processed batch 5/116
  Processed batch 6/116
  Processed batch 7/116
  Processed batch 8/116
  Processed batch 9/116
  Processed batch 10/116
  Processed batch 11/116
  Processed batch 12/116
  Processed batch 13/116
  Processed batch 14/116
  Processed batch 15/116
  Processed batch 16/116
  Processed batch 17/116
  Processed batch 18/116
  Processed batch 19/116
  Processed batch 20/116
  Processed batch 21/116
  Processed batch 22/116
  Processed batch 23/116
  Processe

In [17]:
# Install required packages
!pip install streamlit -q
!pip install google-generativeai -q
!pip install PyPDF2 -q
!pip install python-docx -q
!pip install SpeechRecognition -q
!pip install gTTS -q
!pip install Pillow -q
!pip install pyngrok -q

# Install system dependencies for audio processing
!apt-get update
!apt-get install -y portaudio19-dev python3-pyaudio
!pip install pyaudio

# Install additional audio processing libraries
!apt-get install -y ffmpeg
!pip install pydub -q

!apt-get install ffmpeg
!pip install --quiet streamlit streamlit-audiorecorder git+https://github.com/stlukey/whispercpp.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.13.0 requires google-cloud-aiplatform[agent-engines]<2.0.0,>=1.95.1, but you have google-cloud-aiplatform 1.71.1 which is incompatible.
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/c

In [27]:
!pip install langfuse langgraph -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 18.5 MB/s eta 0:00:00


In [39]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import traceback
import io
import base64
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Optional, TypedDict
from sklearn.metrics.pairwise import cosine_similarity
import time
import uuid
import re

# Langfuse import
from langfuse import Langfuse

# Vertex AI imports
import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from google.cloud import storage

# LangGraph imports
try:
    from langgraph.graph import StateGraph, END
    from langgraph.prebuilt import ToolNode
    from langgraph.checkpoint.memory import MemorySaver
    LANGGRAPH_ENABLED = True
except ImportError:
    LANGGRAPH_ENABLED = False
    st.warning("LangGraph not available. Install langgraph for advanced workflow features.")

# Audio and file processing imports
try:
    import speech_recognition as sr
    from gtts import gTTS
    import PyPDF2
    import docx
    from PIL import Image
    from tempfile import NamedTemporaryFile
    from audiorecorder import audiorecorder
    from whispercpp import Whisper
    AUDIO_ENABLED = True
except ImportError:
    AUDIO_ENABLED = False
    st.warning("Some audio features disabled. Install required packages for full functionality.")

# Page configuration
st.set_page_config(
    page_title="TechCare Solutions Chatbot",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Enhanced CSS for fixed input interface
st.markdown("""
<style>
    /* Clean, minimal styling */
    .stApp {
        background-color: #ffffff;
        color: #333333;
    }

    /* Fixed input container */
    .fixed-input-container {
        position: fixed;
        bottom: 0;
        left: 0;
        right: 0;
        background: white;
        padding: 15px;
        border-top: 2px solid #f0f0f0;
        z-index: 1000;
        box-shadow: 0 -2px 10px rgba(0,0,0,0.1);
    }

    /* Adjust main content to account for fixed input */
    .main-content {
        margin-bottom: 120px;
        padding-bottom: 20px;
    }

    /* Medical code styling */
    .medical-code-box {
        background: #f8f9fa;
        border-left: 4px solid #4CAF50;
        padding: 15px;
        margin: 15px 0;
        border-radius: 8px;
        border: 1px solid #e9ecef;
        font-family: "Courier New", monospace;
    }

    .code-highlight {
        background: #e9ecef;
        color: #006600;
        padding: 2px 6px;
        border-radius: 4px;
        font-weight: bold;
        font-family: "Courier New", monospace;
    }

    /* Voice input feedback */
    .voice-input-feedback {
        background: #f0f8ff;
        border: 2px solid #4CAF50;
        border-radius: 8px;
        padding: 15px;
        margin: 15px 0;
        color: #2d5a2d;
        font-weight: 500;
        animation: pulse 2s infinite;
    }

    @keyframes pulse {
        0% { border-color: #4CAF50; }
        50% { border-color: #81C784; }
        100% { border-color: #4CAF50; }
    }

    /* File info styling */
    .file-info {
        background: #f8f9fa;
        border: 1px solid #dee2e6;
        border-radius: 8px;
        padding: 12px;
        margin: 8px 0;
        font-size: 0.9em;
    }

    /* Audio player styling */
    .audio-player {
        margin: 15px 0;
        padding: 15px;
        background: #f8f9fa;
        border-radius: 8px;
        border: 1px solid #dee2e6;
    }

    .audio-player audio {
        width: 100%;
        height: 45px;
        border-radius: 8px;
    }

    /* FAQ styling */
    .faq-item {
        background: #f8f9fa;
        border: 1px solid #dee2e6;
        border-radius: 6px;
        padding: 10px;
        margin: 5px 0;
        cursor: pointer;
        transition: all 0.2s ease;
    }

    .faq-item:hover {
        background: #e9ecef;
        border-color: #4CAF50;
    }

    /* Voice recording indicator */
    .recording-indicator {
        background: #ffebee;
        border: 2px solid #f44336;
        border-radius: 8px;
        padding: 15px;
        text-align: center;
        animation: recordingPulse 1s infinite;
    }

    @keyframes recordingPulse {
        0% { background-color: #ffebee; }
        50% { background-color: #ffcdd2; }
        100% { background-color: #ffebee; }
    }

    /* Input interface styling */
    .input-interface {
        display: flex;
        gap: 10px;
        align-items: flex-end;
        background: white;
        padding: 10px;
        border-radius: 8px;
        border: 1px solid #dee2e6;
    }

    .voice-button {
        min-width: 60px;
        height: 60px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 20px;
    }

    /* Workflow status styling */
    .workflow-status {
        background: #e3f2fd;
        border: 1px solid #2196f3;
        border-radius: 8px;
        padding: 10px;
        margin: 10px 0;
        font-size: 0.9em;
    }
</style>
""", unsafe_allow_html=True)

# Configuration
PROJECT_ID = "i-monolith-468706-i9"
LOCATION = "us-central1"
BUCKET_NAME = "bucket_jonah"

# Langfuse API Keys
LANGFUSE_SECRET_KEY = "sk-lf-fb61a69a-7805-4423-86fe-71d9cf1ed2aa"
LANGFUSE_PUBLIC_KEY = "pk-lf-7c1d54e4-6bae-40db-ad9b-d469c92eb855"
LANGFUSE_HOST = "https://cloud.langfuse.com"

# Sample queries for FAQ
SAMPLE_QUERIES = [
    "chest pain and shortness of breath",
    "cardiac arrest and heart failure",
    "difficulty breathing",
    "hypotension and shock",
    "mechanical ventilation",
]

# LangGraph State Definition
class WorkflowState(TypedDict):
    """State for the medical RAG workflow"""
    user_query: str
    original_query: str
    processed_query: str
    query_intent: str
    confidence_threshold: float
    top_k: int
    file_context: str
    similar_records: List[Dict[str, Any]]
    medical_codes: Dict[str, Dict]
    rag_response: str
    fallback_response: str
    final_response: str
    use_fallback: bool
    fallback_reason: str
    max_similarity: float
    workflow_status: List[str]
    trace_id: Optional[str]
    error: Optional[str]

@st.cache_resource
def initialize_langfuse():
    """Initialize Langfuse client once and cache it"""
    try:
        langfuse = Langfuse(
            secret_key=LANGFUSE_SECRET_KEY,
            public_key=LANGFUSE_PUBLIC_KEY,
            host=LANGFUSE_HOST
        )
        return langfuse
    except Exception as e:
        st.error(f"Failed to initialize Langfuse: {e}")
        return None

@st.cache_resource
def load_whisper_model():
    """Load Whisper model once and cache it"""
    if AUDIO_ENABLED:
        try:
            return Whisper("tiny")
        except Exception as e:
            st.error(f"Failed to load Whisper model: {e}")
            return None
    return None

class QueryAnalysisAgent:
    """Agent responsible for analyzing and preprocessing user queries"""

    def __init__(self, generative_model):
        self.generative_model = generative_model

    def analyze_query(self, state: WorkflowState) -> WorkflowState:
        """Analyze user query intent and preprocess it"""
        try:
            analysis_prompt = f"""
            Analyze the following medical query and provide:
            1. Query intent (diagnostic, procedural, symptom-based, code-lookup, general)
            2. Key medical terms extracted
            3. Processed query (cleaned and expanded with medical synonyms)

            Original Query: {state['user_query']}

            Provide response in JSON format:
            {{
                "intent": "intent_type",
                "key_terms": ["term1", "term2"],
                "processed_query": "enhanced query with medical terms"
            }}
            """

            response = self.generative_model.generate_content(analysis_prompt)
            try:
                analysis = json.loads(response.text)
                state['query_intent'] = analysis.get('intent', 'general')
                state['processed_query'] = analysis.get('processed_query', state['user_query'])
            except json.JSONDecodeError:
                # Fallback if JSON parsing fails
                state['query_intent'] = 'general'
                state['processed_query'] = state['user_query']

            state['workflow_status'].append(f"✓ Query analyzed - Intent: {state['query_intent']}")

        except Exception as e:
            state['error'] = f"Query analysis error: {str(e)}"
            state['processed_query'] = state['user_query']
            state['query_intent'] = 'general'

        return state

class RetrievalAgent:
    """Agent responsible for retrieving similar medical records"""

    def __init__(self, rag_system):
        self.rag_system = rag_system

    def retrieve_records(self, state: WorkflowState) -> WorkflowState:
        """Retrieve similar medical records based on processed query"""
        try:
            # Prepare enhanced query with file context
            enhanced_query = state['processed_query']
            if state.get('file_context'):
                enhanced_query += f"\n\nAdditional context: {state['file_context'][:1000]}"

            # Search for similar records
            similar_records = self.rag_system.search_similar_records(enhanced_query, state['top_k'])
            state['similar_records'] = similar_records

            if similar_records:
                state['max_similarity'] = max([record['similarity_score'] for record in similar_records])
                state['workflow_status'].append(f"✓ Retrieved {len(similar_records)} records - Max similarity: {state['max_similarity']:.3f}")
            else:
                state['max_similarity'] = 0.0
                state['workflow_status'].append("✓ No similar records found")

        except Exception as e:
            state['error'] = f"Retrieval error: {str(e)}"
            state['similar_records'] = []
            state['max_similarity'] = 0.0

        return state

class MedicalCodingAgent:
    """Agent responsible for extracting and processing medical codes"""

    def __init__(self, rag_system):
        self.rag_system = rag_system

    def extract_codes(self, state: WorkflowState) -> WorkflowState:
        """Extract medical codes from retrieved records"""
        try:
            if state['similar_records']:
                medical_codes = self.rag_system.extract_medical_codes_from_results(state['similar_records'])
                state['medical_codes'] = medical_codes

                total_codes = sum(len(codes) for codes in medical_codes.values())
                state['workflow_status'].append(f"✓ Extracted {total_codes} medical codes")
            else:
                state['medical_codes'] = {'diagnoses': {}, 'procedures': {}, 'cpt_codes': {}}
                state['workflow_status'].append("✓ No medical codes to extract")

        except Exception as e:
            state['error'] = f"Medical coding error: {str(e)}"
            state['medical_codes'] = {'diagnoses': {}, 'procedures': {}, 'cpt_codes': {}}

        return state

class ResponseGenerationAgent:
    """Agent responsible for generating contextual responses"""

    def __init__(self, rag_system):
        self.rag_system = rag_system

    def generate_response(self, state: WorkflowState) -> WorkflowState:
        """Generate appropriate response based on similarity threshold"""
        try:
            if (state['max_similarity'] >= state['confidence_threshold'] and
                state['similar_records'] and
                not state.get('error')):

                # Generate RAG response
                filtered_records = [r for r in state['similar_records']
                                  if r['similarity_score'] >= state['confidence_threshold']]

                rag_response, trace_id = self.rag_system.generate_rag_response(
                    state['processed_query'],
                    filtered_records,
                    state['medical_codes']
                )

                state['rag_response'] = rag_response
                state['final_response'] = f"🏥 **RAG Response** (Max similarity: {state['max_similarity']:.3f})\n\n{rag_response}"
                state['use_fallback'] = False
                state['trace_id'] = trace_id
                state['workflow_status'].append("✓ Generated RAG response")

            else:
                # Generate fallback response
                fallback_reason = (
                    f"Similarity {state['max_similarity']:.3f} below threshold {state['confidence_threshold']:.3f}"
                    if state['similar_records'] else "No similar records found"
                )

                if state.get('error'):
                    fallback_reason = f"Error in workflow: {state['error']}"

                fallback_response, trace_id = self.rag_system.generate_fallback_response(state['original_query'])

                state['fallback_response'] = fallback_response
                state['final_response'] = f"🤖 **Fallback Response** ({fallback_reason})\n\n{fallback_response}"
                state['use_fallback'] = True
                state['fallback_reason'] = fallback_reason
                state['trace_id'] = trace_id
                state['workflow_status'].append("✓ Generated fallback response")

        except Exception as e:
            # Error fallback
            error_response = f"❌ Error generating response: {str(e)}"
            state['final_response'] = error_response
            state['use_fallback'] = True
            state['fallback_reason'] = f"Generation error: {str(e)}"
            state['error'] = str(e)

        return state

class MedicalRAGWorkflow:
    """LangGraph workflow orchestrating the medical RAG process"""

    def __init__(self, rag_system):
        self.rag_system = rag_system

        # Initialize agents
        self.query_agent = QueryAnalysisAgent(rag_system.generative_model)
        self.retrieval_agent = RetrievalAgent(rag_system)
        self.coding_agent = MedicalCodingAgent(rag_system)
        self.response_agent = ResponseGenerationAgent(rag_system)

        # Build workflow
        self.workflow = self._build_workflow()

    def _build_workflow(self):
        """Build the LangGraph workflow"""
        if not LANGGRAPH_ENABLED:
            return None

        # Create workflow
        workflow = StateGraph(WorkflowState)

        # Add nodes
        workflow.add_node("query_analysis", self._query_analysis_node)
        workflow.add_node("retrieval", self._retrieval_node)
        workflow.add_node("medical_coding", self._medical_coding_node)
        workflow.add_node("quality_check", self._quality_check_node)
        workflow.add_node("response_generation", self._response_generation_node)
        workflow.add_node("final_processing", self._final_processing_node)

        # Define workflow edges
        workflow.set_entry_point("query_analysis")
        workflow.add_edge("query_analysis", "retrieval")
        workflow.add_edge("retrieval", "medical_coding")
        workflow.add_edge("medical_coding", "quality_check")
        workflow.add_edge("quality_check", "response_generation")
        workflow.add_edge("response_generation", "final_processing")
        workflow.add_edge("final_processing", END)

        # Compile workflow
        memory = MemorySaver()
        return workflow.compile(checkpointer=memory)

    def _query_analysis_node(self, state: WorkflowState) -> WorkflowState:
        """Node for query analysis"""
        return self.query_agent.analyze_query(state)

    def _retrieval_node(self, state: WorkflowState) -> WorkflowState:
        """Node for document retrieval"""
        return self.retrieval_agent.retrieve_records(state)

    def _medical_coding_node(self, state: WorkflowState) -> WorkflowState:
        """Node for medical code extraction"""
        return self.coding_agent.extract_codes(state)

    def _quality_check_node(self, state: WorkflowState) -> WorkflowState:
        """Node for quality validation of retrieved results"""
        try:
            # Additional quality checks based on query intent
            if state['query_intent'] == 'diagnostic' and state['max_similarity'] > 0:
                # Boost threshold for diagnostic queries
                if state['max_similarity'] < 0.8:
                    state['workflow_status'].append("⚠ Diagnostic query - similarity below recommended threshold")

            elif state['query_intent'] == 'code-lookup':
                # For code lookup, check if we found relevant codes
                total_codes = sum(len(codes) for codes in state['medical_codes'].values())
                if total_codes == 0 and state['max_similarity'] < 0.6:
                    state['workflow_status'].append("⚠ Code lookup - no relevant codes found")

            # Quality score based on similarity and intent match
            quality_score = state['max_similarity']
            if state['query_intent'] in ['diagnostic', 'procedural'] and quality_score > 0.7:
                quality_score += 0.1  # Boost for clinical queries with good similarity

            state['workflow_status'].append(f"✓ Quality check completed - Score: {quality_score:.3f}")

        except Exception as e:
            state['error'] = f"Quality check error: {str(e)}"

        return state

    def _response_generation_node(self, state: WorkflowState) -> WorkflowState:
        """Node for response generation"""
        return self.response_agent.generate_response(state)

    def _final_processing_node(self, state: WorkflowState) -> WorkflowState:
        """Node for final processing and cleanup"""
        try:
            # Add workflow completion status
            state['workflow_status'].append("✓ Workflow completed successfully")

            # Ensure all required fields are present
            if not state.get('final_response'):
                state['final_response'] = "❌ No response generated"
                state['use_fallback'] = True
                state['fallback_reason'] = "Final processing error"

        except Exception as e:
            state['error'] = f"Final processing error: {str(e)}"
            state['final_response'] = f"❌ Workflow error: {str(e)}"

        return state

    def execute(self, user_query: str, confidence_threshold: float, top_k: int, file_context: str = "") -> Dict[str, Any]:
        """Execute the workflow"""
        if not self.workflow:
            # Fallback to original method if LangGraph not available
            return self._execute_fallback(user_query, confidence_threshold, top_k, file_context)

        try:
            # Initialize state
            initial_state = WorkflowState(
                user_query=user_query,
                original_query=user_query,
                processed_query=user_query,
                query_intent='general',
                confidence_threshold=confidence_threshold,
                top_k=top_k,
                file_context=file_context,
                similar_records=[],
                medical_codes={'diagnoses': {}, 'procedures': {}, 'cpt_codes': {}},
                rag_response='',
                fallback_response='',
                final_response='',
                use_fallback=False,
                fallback_reason='',
                max_similarity=0.0,
                workflow_status=['🚀 Workflow started'],
                trace_id=None,
                error=None
            )

            # Execute workflow
            config = {"configurable": {"thread_id": str(uuid.uuid4())}}
            result = self.workflow.invoke(initial_state, config)

            # Return formatted result
            return {
                'success': True,
                'use_fallback': result['use_fallback'],
                'fallback_reason': result.get('fallback_reason', ''),
                'response': result['final_response'],
                'medical_codes': result.get('medical_codes', {}),
                'max_similarity': result.get('max_similarity', 0.0),
                'total_records_found': len(result.get('similar_records', [])),
                'records_above_threshold': len([r for r in result.get('similar_records', [])
                                              if r.get('similarity_score', 0) >= confidence_threshold]),
                'trace_id': result.get('trace_id'),
                'workflow_status': result.get('workflow_status', []),
                'query_intent': result.get('query_intent', 'general')
            }

        except Exception as e:
            return {
                'success': False,
                'error': f"Workflow execution error: {str(e)}",
                'use_fallback': True,
                'trace_id': None
            }

    def _execute_fallback(self, user_query: str, confidence_threshold: float, top_k: int, file_context: str = "") -> Dict[str, Any]:
        """Fallback execution when LangGraph is not available"""
        return self.rag_system.query_with_threshold(user_query, confidence_threshold, top_k)

class IntegratedMedicalRAG:
    """Enhanced Medical RAG system with multi-modal input support and Langfuse tracking"""

    def __init__(self, project_id: str, location: str, bucket_name: str, langfuse: Optional[Langfuse] = None):
        self.project_id = project_id
        self.location = location
        self.bucket_name = bucket_name
        self.langfuse = langfuse

        try:
            # Initialize Vertex AI
            vertexai.init(project=project_id, location=location)
            self.embedding_model = TextEmbeddingModel.from_pretrained('text-embedding-004')
            self.generative_model = GenerativeModel('gemini-2.5-flash')
            self.storage_client = storage.Client(project=project_id)
            self.bucket = self.storage_client.bucket(bucket_name)

            self.medical_records = []
            self.is_initialized = True

            # Initialize LangGraph workflow
            self.workflow_engine = MedicalRAGWorkflow(self) if LANGGRAPH_ENABLED else None

        except Exception as e:
            st.error(f"Failed to initialize Vertex AI: {e}")
            self.is_initialized = False

    def load_embeddings_from_gcs(self):
        """Load embeddings from GCS"""
        try:
            gcs_path = "embeddings/medical_embeddings.jsonl"
            blob = self.bucket.blob(gcs_path)
            local_path = "/tmp/medical_embeddings.jsonl"
            blob.download_to_filename(local_path)

            self.medical_records = []
            with open(local_path, 'r') as f:
                for line in f:
                    record = json.loads(line.strip())
                    self.medical_records.append(record)

            return True, f"Loaded {len(self.medical_records)} medical records"
        except Exception as e:
            return False, f"Error loading embeddings: {e}"

    def search_similar_records(self, query: str, top_k: int = 10):
        """Search for similar medical records"""
        if not self.medical_records:
            return []

        try:
            # Generate query embedding
            query_embeddings = self.embedding_model.get_embeddings([query])
            query_vector = np.array(query_embeddings[0].values)

            # Get all record embeddings
            record_embeddings = np.array([record['embedding'] for record in self.medical_records])

            # Calculate similarities
            similarities = cosine_similarity([query_vector], record_embeddings)[0]

            # Get top-k similar records
            top_indices = np.argsort(similarities)[-top_k:][::-1]

            similar_records = []
            for idx in top_indices:
                record = self.medical_records[idx].copy()
                record['similarity_score'] = float(similarities[idx])
                similar_records.append(record)

            return similar_records
        except Exception as e:
            st.error(f"Error in similarity search: {e}")
            return []

    def extract_medical_codes_from_results(self, similar_records: List[Dict[str, Any]]):
        """Extract medical codes from search results"""
        medical_codes = {
            'diagnoses': {},
            'procedures': {},
            'cpt_codes': {}
        }

        for record in similar_records:
            # Process diagnoses
            for diag in record.get('diagnoses', []):
                code = diag.get('ICD9_CODE')
                if code:
                    if code not in medical_codes['diagnoses']:
                        medical_codes['diagnoses'][code] = {
                            'short_title': diag.get('SHORT_TITLE', ''),
                            'long_title': diag.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['diagnoses'][code]['frequency'] += 1
                    medical_codes['diagnoses'][code]['max_similarity'] = max(
                        medical_codes['diagnoses'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process procedures
            for proc in record.get('procedures', []):
                code = str(proc.get('ICD9_CODE'))
                if code and code != "None":
                    if code not in medical_codes['procedures']:
                        medical_codes['procedures'][code] = {
                            'short_title': proc.get('SHORT_TITLE', ''),
                            'long_title': proc.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['procedures'][code]['frequency'] += 1
                    medical_codes['procedures'][code]['max_similarity'] = max(
                        medical_codes['procedures'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process CPT codes
            for cpt in record.get('cpt_codes', []):
                code = cpt.get('CPT_CD')
                if code:
                    if code not in medical_codes['cpt_codes']:
                        medical_codes['cpt_codes'][code] = {
                            'description': cpt.get('DESCRIPTION', ''),
                            'section': cpt.get('SECTIONHEADER', ''),
                            'subsection': cpt.get('SUBSECTIONHEADER', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['cpt_codes'][code]['frequency'] += 1
                    medical_codes['cpt_codes'][code]['max_similarity'] = max(
                        medical_codes['cpt_codes'][code]['max_similarity'],
                        record['similarity_score']
                    )

        return medical_codes

    def generate_rag_response(self, query: str, similar_records: List[Dict[str, Any]], medical_codes: Dict[str, Dict]):
        """Generate response using RAG with Langfuse tracking"""
        start_time = time.time()

        # Prepare context from top records
        context = "\n\n".join([f"Record {i+1}: {record['content'][:300]}..."
                                for i, record in enumerate(similar_records[:3])])

        # Format medical codes
        diagnoses_text = ""
        if medical_codes['diagnoses']:
            diagnoses_text = "\nDIAGNOSIS CODES FOUND:\n"
            for code, info in sorted(medical_codes['diagnoses'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                diagnoses_text += f"- {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info['max_similarity']:.3f})\n"

        procedures_text = ""
        if medical_codes['procedures']:
            procedures_text = "\nPROCEDURE CODES FOUND:\n"
            for code, info in sorted(medical_codes['procedures'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                procedures_text += f"- {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info['max_similarity']:.3f})\n"

        cpt_text = ""
        if medical_codes['cpt_codes']:
            cpt_text = "\nCPT CODES FOUND:\n"
            for code, info in sorted(medical_codes['cpt_codes'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                cpt_text += f"- {code}: {info['description']} (freq: {info['frequency']}, sim: {info['max_similarity']:.3f})\n"

        prompt = f"""You are a medical coding assistant. Based on the following medical records context and the user's query, provide relevant medical codes and explanations.

CONTEXT FROM MEDICAL RECORDS:
{context}

{diagnoses_text}
{procedures_text}
{cpt_text}

USER QUERY: {query}

Please provide a comprehensive response that:
1. Explains what medical codes are most relevant to the query
2. Provides context on why these codes match the query
3. Lists the specific ICD-9 diagnosis codes, procedure codes, and CPT codes
4. Explains what each code represents in simple terms
5. Notes the similarity scores to show relevance

Keep your response concise and well-formatted."""

        try:
            if self.langfuse:
                with self.langfuse.start_as_current_span(name="rag-response", input={"user_query": query}) as span:
                    with self.langfuse.start_as_current_generation(
                        name="gemini-rag-generation",
                        model="gemini-2.5-flash",
                        input={"prompt": prompt}
                    ) as generation:
                        response = self.generative_model.generate_content(prompt)
                        response_text = response.text
                        processing_time = time.time() - start_time

                        generation.update(output=response_text)
                        span.update_trace(output={
                            "response": response_text,
                            "processing_time": processing_time,
                            "response_type": "RAG",
                            "records_used": len(similar_records),
                            "medical_codes_found": sum(len(codes) for codes in medical_codes.values())
                        })

                        return response_text, span.trace_id
            else:
                response = self.generative_model.generate_content(prompt)
                return response.text, None
        except Exception as e:
            error_msg = f"Error generating RAG response: {e}"
            if self.langfuse:
                with self.langfuse.start_as_current_span(name="rag-response-error", input={"user_query": query}) as span:
                    span.update_trace(output={"error": error_msg})
                    return error_msg, span.trace_id
            return error_msg, None

    def generate_fallback_response(self, query: str):
        """Generate fallback response using Gemini when confidence is low with Langfuse tracking"""
        start_time = time.time()

        prompt = f"""You are a medical coding assistant. The user has asked about: {query}

Since no highly relevant medical records were found in the database, please provide a general medical coding response that includes:

1. Common ICD-9 diagnosis codes that might be related to this query
2. Typical CPT codes for procedures related to this condition
3. Brief explanations of what these codes represent
4. General medical context for the query

Please format your response clearly with:
- DIAGNOSIS CODES (ICD-9): List relevant codes with descriptions
- PROCEDURE CODES: List relevant CPT codes with descriptions
- MEDICAL CONTEXT: Brief explanation of the condition/symptoms

Remember to advise consultation with healthcare professionals for serious medical concerns."""

        try:
            if self.langfuse:
                with self.langfuse.start_as_current_span(name="fallback-response", input={"user_query": query}) as span:
                    with self.langfuse.start_as_current_generation(
                        name="gemini-fallback-generation",
                        model="gemini-2.5-flash",
                        input={"prompt": prompt}
                    ) as generation:
                        response = self.generative_model.generate_content(prompt)
                        response_text = response.text
                        processing_time = time.time() - start_time

                        generation.update(output=response_text)
                        span.update_trace(output={
                            "response": response_text,
                            "processing_time": processing_time,
                            "response_type": "Fallback"
                        })

                        return response_text, span.trace_id
            else:
                response = self.generative_model.generate_content(prompt)
                return response.text, None
        except Exception as e:
            error_msg = f"Error generating fallback response: {e}"
            if self.langfuse:
                with self.langfuse.start_as_current_span(name="fallback-response-error", input={"user_query": query}) as span:
                    span.update_trace(output={"error": error_msg})
                    return error_msg, span.trace_id
            return error_msg, None

    def query_with_threshold(self, user_query: str, confidence_threshold: float, top_k: int = 10):
        """Main query function with confidence threshold and Langfuse tracking"""
        if not self.is_initialized:
            return {
                'success': False,
                'error': "RAG system not initialized",
                'use_fallback': True,
                'trace_id': None
            }

        # Use LangGraph workflow if available
        if self.workflow_engine:
            file_context = st.session_state.get('uploaded_files_text', '')
            return self.workflow_engine.execute(user_query, confidence_threshold, top_k, file_context)

        # Fallback to original implementation
        return self._original_query_implementation(user_query, confidence_threshold, top_k)

    def _original_query_implementation(self, user_query: str, confidence_threshold: float, top_k: int = 10):
        """Original query implementation as fallback"""
        # Search for similar records
        similar_records = self.search_similar_records(user_query, top_k)
        if not similar_records:
            response, trace_id = self.generate_fallback_response(user_query)
            return {
                'success': True,
                'use_fallback': True,
                'fallback_reason': "No similar records found",
                'response': response,
                'trace_id': trace_id
            }

        # Check highest similarity score against threshold
        max_similarity = max([record['similarity_score'] for record in similar_records])
        if max_similarity < confidence_threshold:
            response, trace_id = self.generate_fallback_response(user_query)
            return {
                'success': True,
                'use_fallback': True,
                'fallback_reason': f"Highest similarity {max_similarity:.3f} below threshold {confidence_threshold:.3f}",
                'response': response,
                'max_similarity': max_similarity,
                'trace_id': trace_id
            }

        # Use RAG response
        filtered_records = [r for r in similar_records if r['similarity_score'] >= confidence_threshold]
        medical_codes = self.extract_medical_codes_from_results(filtered_records)
        response, trace_id = self.generate_rag_response(user_query, filtered_records, medical_codes)

        return {
            'success': True,
            'use_fallback': False,
            'response': response,
            'medical_codes': medical_codes,
            'filtered_records': filtered_records,
            'max_similarity': max_similarity,
            'total_records_found': len(similar_records),
            'records_above_threshold': len(filtered_records),
            'trace_id': trace_id
        }

# File processing functions
def extract_text_from_pdf(file_path: str) -> str:
    """Extract text from PDF file"""
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text.strip()
    except Exception as e:
        st.error(f"Error reading PDF: {e}")
        return ""

def extract_text_from_docx(file_path: str) -> str:
    """Extract text from DOCX file"""
    try:
        doc = docx.Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text
        return text.strip()
    except Exception as e:
        st.error(f"Error reading DOCX: {e}")
        return ""

def read_text_file_with_fallback(file_path: str) -> str:
    """Read text file with multiple encoding fallbacks"""
    encodings_to_try = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252', 'iso-8859-1']

    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                content = f.read().strip()
                if content:  # Return if content is found
                    return content
        except (UnicodeDecodeError, Exception):
            continue

    # If all encodings fail, try binary read
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            decoded_content = content.decode('utf-8', errors='ignore').strip()
            if decoded_content:
                return decoded_content
    except Exception:
        pass

    return ""  # Return empty string instead of error message

def process_single_file(file) -> dict:
    """Process a single uploaded file and return file info with extracted text"""
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(file.name)[1]) as tmp_file:
            tmp_file.write(file.getvalue())
            tmp_file_path = tmp_file.name

        file_extension = os.path.splitext(file.name)[1].lower()
        text = ""
        file_type = "Unknown"

        if file.type == "application/pdf" or file_extension == ".pdf":
            text = extract_text_from_pdf(tmp_file_path)
            file_type = "PDF"
        elif file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document" or file_extension == ".docx":
            text = extract_text_from_docx(tmp_file_path)
            file_type = "Word Document"
        elif (file.type and file.type.startswith("text")) or file_extension in [".txt", ".text", ".log", ".md", ".csv"]:
            text = read_text_file_with_fallback(tmp_file_path)
            file_type = "Text File"
        else:
            # Fallback: try to read as text file
            text = read_text_file_with_fallback(tmp_file_path)
            if text and len(text.strip()) > 0:
                file_type = "Text File (Auto-detected)"
            else:
                text = f"Unsupported file type: {file.name}"
                file_type = "Unsupported"

        os.unlink(tmp_file_path)

        status = "success" if text and text.strip() and not text.startswith("Unsupported") else "failed"

        return {
            "name": file.name,
            "type": file_type,
            "size": len(file.getvalue()),
            "text": text,
            "status": status,
            "mimetype": file.type if hasattr(file, "type") else "unknown",
            "extension": file_extension,
            "text_length": len(text) if text else 0
        }
    except Exception as e:
        return {
            "name": file.name,
            "type": "Unknown",
            "size": 0,
            "text": "",
            "status": "error",
            "error": str(e),
            "mimetype": file.type if hasattr(file, "type") else "unknown",
            "extension": os.path.splitext(file.name)[1].lower() if hasattr(file, "name") else "unknown",
            "text_length": 0
        }

def text_to_speech_base64(text: str, language: str = "en") -> str:
    """Convert text to speech and return base64 encoded audio"""
    try:
        if len(text) > 500:
            text_for_tts = text[:500] + "..."
        else:
            text_for_tts = text

        tts = gTTS(text=text_for_tts, lang=language, slow=False)

        with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as tmp_file:
            tts.save(tmp_file.name)

            with open(tmp_file.name, 'rb') as audio_file:
                audio_bytes = audio_file.read()
                audio_base64 = base64.b64encode(audio_bytes).decode()

            os.unlink(tmp_file.name)
            return audio_base64

    except Exception as e:
        st.error(f"Error generating speech: {e}")
        return None

def improved_voice_transcription(audio):
    """Improved voice transcription with better error handling"""
    try:
        if not AUDIO_ENABLED:
            return "Audio features not available"

        if st.session_state.whisper_model is None:
            st.session_state.whisper_model = load_whisper_model()
            if st.session_state.whisper_model is None:
                return "Whisper model not available"

        w = st.session_state.whisper_model

        # Create temporary file with proper cleanup
        with NamedTemporaryFile(suffix=".wav", delete=False) as temp:
            # Export as WAV for better compatibility
            audio.export(temp.name, format="wav")

            # Transcribe
            result = w.transcribe(temp.name)
            text = w.extract_text(result)

            # Clean up temp file
            os.unlink(temp.name)

            # Return first result or empty if none
            if text and len(text) > 0:
                transcribed = text[0].strip()
                # Filter out common transcription artifacts
                if len(transcribed) > 3 and not transcribed.lower().startswith("you"):
                    return transcribed
                else:
                    return "Please speak more clearly and try again"
            else:
                return "No speech detected. Please try again"

    except Exception as e:
        st.error(f"Transcription error: {e}")
        return f"Transcription failed: {str(e)}"

def highlight_medical_codes(text: str) -> str:
    """Highlight medical codes in the response text"""
    # Pattern to match medical codes (ICD-9, CPT, etc.)
    patterns = [
        r'\b\d{1,3}\.\d{1,2}\b',  # ICD-9 codes like 250.00
        r'\b\d{5}\b',  # CPT codes
        r'\b[A-Z]\d{2}\b',  # Some ICD codes
        r'\b\d{3}-\d{3}-\d{4}\b'  # Other medical codes
    ]

    highlighted_text = text
    for pattern in patterns:
        highlighted_text = re.sub(pattern, lambda m: f'<span class="code-highlight">{m.group()}</span>', highlighted_text)

    return highlighted_text

def display_medical_codes_box(medical_codes: Dict[str, Dict]):
    """Display medical codes in a highlighted box"""
    if not medical_codes or not any(medical_codes.values()):
        return

    codes_html = '<div class="medical-code-box"><h4>📋 Medical Codes Found</h4>'

    # Diagnosis codes
    if medical_codes.get('diagnoses'):
        codes_html += '<h5>🏥 Diagnosis Codes (ICD-9)</h5>'
        for code, info in sorted(medical_codes['diagnoses'].items(),
                               key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
            codes_html += f'<p><span class="code-highlight">{code}</span>: {info["long_title"]}<br><small>Frequency: {info["frequency"]}, Similarity: {info["max_similarity"]:.3f}</small></p>'

    # Procedure codes
    if medical_codes.get('procedures'):
        codes_html += '<h5>⚕️ Procedure Codes (ICD-9)</h5>'
        for code, info in sorted(medical_codes['procedures'].items(),
                               key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
            codes_html += f'<p><span class="code-highlight">{code}</span>: {info["long_title"]}<br><small>Frequency: {info["frequency"]}, Similarity: {info["max_similarity"]:.3f}</small></p>'

    # CPT codes
    if medical_codes.get('cpt_codes'):
        codes_html += '<h5>💊 CPT Codes</h5>'
        for code, info in sorted(medical_codes['cpt_codes'].items(),
                               key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
            codes_html += f'<p><span class="code-highlight">{code}</span>: {info["description"]}<br><small>Frequency: {info["frequency"]}, Similarity: {info["max_similarity"]:.3f}</small></p>'

    codes_html += '</div>'
    st.markdown(codes_html, unsafe_allow_html=True)

def display_workflow_status(workflow_status: List[str]):
    """Display workflow execution status"""
    if workflow_status and LANGGRAPH_ENABLED:
        status_html = '<div class="workflow-status"><h5>🔄 Workflow Status</h5><ul>'
        for status in workflow_status:
            status_html += f'<li>{status}</li>'
        status_html += '</ul></div>'
        st.markdown(status_html, unsafe_allow_html=True)


def execute_rag_query(prompt, confidence_threshold, top_k):
    """Execute RAG query and handle the response"""
    if not st.session_state.rag_system or not st.session_state.embeddings_loaded:
        error_msg = "⚠️ RAG system not ready. Please wait for initialization or check the sidebar."
        st.error(error_msg)
        st.session_state.messages.append({"role": "assistant", "content": error_msg})
        return

    with st.spinner("🔥 Processing your query..."):
        # Get RAG response (now with LangGraph workflow)
        result = st.session_state.rag_system.query_with_threshold(
            prompt,
            confidence_threshold,
            top_k
        )

        if not result['success']:
            response = f"❌ Error: {result['error']}"
            st.error(response)
            st.session_state.messages.append({"role": "assistant", "content": response})
        elif result['use_fallback']:
            response = result['response']   # already formatted in workflow
            st.markdown(response)


            # Display workflow status if available
            if result.get('workflow_status'):
                display_workflow_status(result['workflow_status'])

            st.session_state.messages.append({
                "role": "assistant",
                "content": response,
                "trace_id": result.get('trace_id'),
                "workflow_status": result.get('workflow_status', [])
            })

            if result.get('trace_id') and st.session_state.langfuse:
                trace_url = f"https://cloud.langfuse.com/project/cmfiuk7380001ad074uio1la1/traces/{result['trace_id']}"
                st.markdown(f"[→ View Langfuse Trace]({trace_url})")
        else:
            response = result['response']   # already formatted in workflow
            highlighted_response = highlight_medical_codes(response)
            st.markdown(highlighted_response, unsafe_allow_html=True)

            if result.get('medical_codes'):
                display_medical_codes_box(result['medical_codes'])

            # Display workflow status if available
            if result.get('workflow_status'):
                display_workflow_status(result['workflow_status'])

            st.session_state.messages.append({
                "role": "assistant",
                "content": response,
                "medical_codes": result.get('medical_codes', {}),
                "similarity_score": result['max_similarity'],
                "trace_id": result.get('trace_id'),
                "workflow_status": result.get('workflow_status', []),
                "query_intent": result.get('query_intent', 'general')
            })

            st.info(f"📊 Found {result['total_records_found']} similar records, {result['records_above_threshold']} above threshold")

            # Show query intent if available
            if result.get('query_intent') and result['query_intent'] != 'general':
                st.info(f"🎯 Query Intent: {result['query_intent']}")

            if result.get('trace_id') and st.session_state.langfuse:
                trace_url = f"https://cloud.langfuse.com/project/cmfiuk7380001ad074uio1la1/traces/{result['trace_id']}"
                st.markdown(f"[→ View Langfuse Trace]({trace_url})")


def check_documents_available():
    """Check if documents are available and return status"""
    if not st.session_state.sidebar_files_processed:
        return False, 0, ""

    successful_files = [f for f in st.session_state.sidebar_files_processed.values() if f["status"] == "success"]

    if not successful_files:
        return False, 0, ""

    # Check if the text still exists
    combined_text = ""
    for file_info in successful_files:
        if file_info["text"] and not file_info["text"].startswith("Unsupported") and not file_info["text"].startswith("Error"):
            combined_text += f"\n\n--- Content from {file_info['name']} ---\n{file_info['text']}"

    # Update session state with current combined text
    st.session_state.uploaded_files_text = combined_text

    return len(combined_text.strip()) > 0, len(successful_files), combined_text


# Initialize session state with improved structure
session_state_vars = [
    ('messages', []),
    ('rag_system', None),
    ('embeddings_loaded', False),
    ('whisper_model', None),
    ('uploaded_files_text', ''),
    ('current_uploaded_files', []),
    ('sidebar_files_processed', {}),
    ('voice_transcribed_text', ''),
    ('is_recording', False),
    ('audio_processed', False),
    ('selected_sample_query', ''),
    ('chat_input_key', 0),
    ('pending_query', ''),
    ('execute_pending', False),
    ('langfuse', None)
]

for var, default_value in session_state_vars:
    if var not in st.session_state:
        st.session_state[var] = default_value


def load_rag_system():
    """Initialize and load the RAG system"""
    if st.session_state.rag_system is None:
        with st.spinner("🔥 Initializing RAG system..."):
            st.session_state.rag_system = IntegratedMedicalRAG(
                PROJECT_ID,
                LOCATION,
                BUCKET_NAME,
                st.session_state.langfuse
            )

    if not st.session_state.embeddings_loaded and st.session_state.rag_system.is_initialized:
        with st.spinner("📊 Loading medical embeddings from GCS..."):
            success, message = st.session_state.rag_system.load_embeddings_from_gcs()
            st.session_state.embeddings_loaded = success
            if success:
                st.success(message)
            else:
                st.error(message)


def handle_sample_query_selection(query):
    """Handle sample query selection from FAQ - directly execute RAG"""
    st.session_state.pending_query = query
    st.session_state.execute_pending = True
    st.rerun()


def handle_voice_input():
    """Handle voice input - directly execute RAG"""
    if st.session_state.voice_transcribed_text:
        query = st.session_state.voice_transcribed_text
        st.session_state.voice_transcribed_text = ""
        st.session_state.audio_processed = True

        # Add user message
        st.session_state.messages.append({"role": "user", "content": query})

        with st.chat_message("user"):
            st.markdown(query)

        st.session_state.pending_query = query
        st.session_state.execute_pending = True
        st.rerun()


# Initialize Langfuse client
if st.session_state.langfuse is None:
    st.session_state.langfuse = initialize_langfuse()

# Initialize RAG system automatically
if st.session_state.rag_system is None:
    load_rag_system()


# Main App Layout
st.title("🏥 TechCare Solutions Chatbot")
st.markdown("Your AI-powered medical assistant with advanced medical coding capabilities")

# # Display LangGraph status
# if LANGGRAPH_ENABLED:
#     st.success("✅ LangGraph Workflow Engine Active")
# else:
#     st.warning("⚠️ LangGraph not available - using fallback mode")

# Main content container
with st.container():
    st.markdown('<div class="main-content">', unsafe_allow_html=True)

    # Sidebar Configuration
    with st.sidebar:
        # st.header("⚙️ Configuration")

        # # Display system status
        # if st.session_state.rag_system:
        #     if st.session_state.embeddings_loaded:
        #         st.success("✅ RAG System Ready")
        #         st.info(f"📊 Records loaded: {len(st.session_state.rag_system.medical_records)}")

        #         # LangGraph status in sidebar
        #         if LANGGRAPH_ENABLED and st.session_state.rag_system.workflow_engine:
        #             st.success("🔄 LangGraph Workflow Active")
        #         else:
        #             st.warning("⚠️ Using Legacy Workflow")
        #     else:
        #         st.warning("⚠️ RAG System initialized but embeddings not loaded")
        #         if st.button("🔥 Reload Embeddings", type="primary"):
        #             load_rag_system()
        # else:
        #     st.error("❌ RAG System not initialized")
        #     if st.button("🚀 Initialize RAG System", type="primary"):
        #         load_rag_system()

        # # Langfuse status
        # if st.session_state.langfuse:
        #     st.success("✅ Langfuse Connected")
        # else:
        #     st.error("❌ Langfuse Not Connected")

        # st.divider()

        # Query settings
        st.subheader("🎛️ Query Settings")
        confidence_threshold = st.slider(
            "Confidence Threshold",
            min_value=0.0,
            max_value=1.0,
            value=0.7,
            step=0.01,
            help="Minimum similarity score required to use RAG. Lower scores will fallback to Gemini."
        )

        top_k = st.slider(
            "Max Records to Retrieve",
            min_value=1,
            max_value=10,
            value=3,
            step=5
        )

        st.divider()

        # FAQ Section with Sample Queries
        st.header("❓ FAQ - Sample Queries")
        st.markdown("Click on any query to execute it:")

        for query in SAMPLE_QUERIES:
            if st.button(f"💬 {query}", key=f"faq_{query}", use_container_width=True):
                handle_sample_query_selection(query)

        st.divider()

        # Document Upload Section
        st.header("📄 Document Context")
        st.markdown("Upload medical documents for additional context")

        uploaded_files = st.file_uploader(
            "Choose files to add context",
            accept_multiple_files=True,
            type=['pdf', 'docx', 'txt'],
            key="sidebar_file_uploader",
            help="Upload PDF, Word documents, or text files"
        )

        # Process and display uploaded files
        if uploaded_files:
            current_file_names = [f.name for f in uploaded_files]

            # Check if files have changed
            if set(current_file_names) != set(st.session_state.sidebar_files_processed.keys()):
                with st.spinner("📋 Processing documents..."):
                    st.session_state.sidebar_files_processed = {}
                    combined_text = ""

                    for file in uploaded_files:
                        file_info = process_single_file(file)
                        st.session_state.sidebar_files_processed[file.name] = file_info

                        if file_info["text"] and not file_info["text"].startswith("Unsupported") and not file_info["text"].startswith("Error"):
                            combined_text += f"\n\n--- Content from {file.name} ---\n{file_info['text']}"

                    st.session_state.uploaded_files_text = combined_text

            # Display file information
            st.subheader("📃 Uploaded Documents")
            for filename, file_info in st.session_state.sidebar_files_processed.items():
                if file_info["status"] == "success":
                    with st.expander(f"📄 {filename}", expanded=False):
                        st.markdown(f"**Type:** {file_info['type']}")
                        st.markdown(f"**Size:** {file_info['size']} bytes")
                        st.markdown(f"**Text Length:** {file_info['text_length']} characters")

                        if file_info["text"] and len(file_info["text"]) > 100:
                            st.text_area("Content Preview", file_info["text"][:500] + "...", height=100)
                else:
                    st.error(f"❌ Failed to process: {filename}")
        else:
            # Clear files if no files are uploaded
            if st.session_state.sidebar_files_processed:
                st.session_state.sidebar_files_processed = {}
                st.session_state.uploaded_files_text = ""

        # Langfuse section
        st.divider()
        st.header("📊 Langfuse Tracking")
        if st.session_state.langfuse:
            st.markdown("**Status:** Connected ✅")
            st.markdown("**Tracking:** All responses tracked")
            langfuse_traces_url = "https://cloud.langfuse.com/project/cmfiuk7380001ad074uio1la1/traces"
            st.markdown(f"[View All Traces]({langfuse_traces_url})")
        else:
            st.markdown("**Status:** Disconnected ❌")

    # Execute pending query if available
    if st.session_state.execute_pending and st.session_state.pending_query:
        user_query = st.session_state.pending_query

        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": user_query})

        with st.chat_message("user"):
            st.markdown(user_query)

        # Execute RAG query (real response will replace processing bubble in next rerun)
        execute_rag_query(user_query, confidence_threshold, top_k)

        # Clear pending query
        st.session_state.pending_query = ""
        st.session_state.execute_pending = False
        st.rerun()

    # Display chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            if message["role"] == "assistant":
                # Display the response
                st.markdown(message["content"])

                # Display medical codes box if available
                if message.get("medical_codes"):
                    display_medical_codes_box(message["medical_codes"])

                # Display workflow status if available
                if message.get("workflow_status"):
                    display_workflow_status(message["workflow_status"])

                # Display query intent if available
                if message.get("query_intent") and message["query_intent"] != 'general':
                    st.info(f"🎯 Query Intent: {message['query_intent']}")

                # Display Langfuse trace link if available
                if message.get("trace_id") and st.session_state.langfuse:
                    trace_url = f"https://cloud.langfuse.com/project/cmfiuk7380001ad074uio1la1/traces/{message['trace_id']}"
                    st.markdown(f"[→ View Langfuse Trace]({trace_url})")

                # Always add audio playback for assistant responses (RAG + Gemini fallback)
                if AUDIO_ENABLED:
                    try:
                        audio_base64 = text_to_speech_base64(message["content"][:300])
                        if audio_base64:
                            audio_html = f"""
                            <div class="audio-player">
                                <audio controls>
                                    <source src="data:audio/mp3;base64,{audio_base64}" type="audio/mp3">
                                    Your browser does not support the audio element.
                                </audio>
                            </div>
                            """
                            st.markdown(audio_html, unsafe_allow_html=True)
                    except Exception:
                        pass

            else:
                st.markdown(message["content"])

    # Voice transcription feedback - improved to directly execute
    if st.session_state.voice_transcribed_text and not st.session_state.audio_processed:
        st.markdown(f"""
        <div class="voice-input-feedback">
            <strong>🎤 Voice Input Transcribed:</strong><br>
            "{st.session_state.voice_transcribed_text}"
        </div>
        """, unsafe_allow_html=True)

        col1, col2, col3 = st.columns([2, 1, 1])
        with col1:
            if st.button("📤 Send Voice Message", key="send_voice", type="primary"):
                handle_voice_input()
        with col2:
            if st.button("🗑️ Clear", key="clear_voice"):
                st.session_state.voice_transcribed_text = ""
                st.session_state.audio_processed = True
                st.rerun()
        with col3:
            if st.button("🎤 Record Again", key="record_again"):
                st.session_state.voice_transcribed_text = ""
                st.session_state.audio_processed = True
                st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# Fixed input interface at bottom
with st.container():
    # Context indicator - improved checking
    docs_available, num_docs, combined_text = check_documents_available()
    if docs_available:
        st.info(f"🔍 Context active: {num_docs} document(s) will be included in responses")

    # Input interface
    col1, col2 = st.columns([9, 1])

    with col1:
        # Handle sample query selection
        if st.session_state.selected_sample_query:
            prompt = st.chat_input(
                st.session_state.selected_sample_query if st.session_state.selected_sample_query else "Ask a medical question...",
                key=f"chat_input_{st.session_state.chat_input_key}"
            )
            st.session_state.selected_sample_query = ""
        else:
            prompt = st.chat_input("Ask a medical question...", key=f"chat_input_{st.session_state.chat_input_key}")

    with col2:
        if AUDIO_ENABLED and not st.session_state.voice_transcribed_text:
            audio = audiorecorder(
                start_prompt="🎤",
                stop_prompt="🔴",
                key="voice_recorder",
                show_visualizer=True
            )

            # Handle voice input
            if len(audio) > 0 and not st.session_state.is_recording:
                st.session_state.is_recording = True
                with st.spinner("🎤 Transcribing audio..."):
                    transcribed_text = improved_voice_transcription(audio)
                    if transcribed_text and not transcribed_text.startswith("Error") and not transcribed_text.startswith("Audio features") and not transcribed_text.startswith("Whisper model"):
                        st.session_state.voice_transcribed_text = transcribed_text
                        st.session_state.audio_processed = False
                        st.success("🎤 Voice transcribed successfully!")
                    else:
                        st.error(f"❌ {transcribed_text}")

                st.session_state.is_recording = False
                st.rerun()
        else:
            st.markdown("🔇 Audio disabled")

# Process text input message
if prompt:
    # Reset states
    st.session_state.audio_processed = True
    st.session_state.voice_transcribed_text = ""
    st.session_state.chat_input_key += 1

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate assistant response
    with st.chat_message("assistant"):
        execute_rag_query(prompt, confidence_threshold, top_k)

    # Rerun to refresh the interface
    st.rerun()

# Footer
st.markdown("---")
st.markdown("⚠️ **Disclaimer:** This is a demonstration system. Always consult healthcare professionals for medical advice.")

# Sidebar tracking info
with st.sidebar:
    if st.button("🗑️ Clear Chat"):
        st.session_state.messages = []
        st.rerun()

Overwriting app.py


In [20]:
ngrok_token = "328GeQwD1llQcGkxP3EPmr1QS6t_2JS19i8o5SqhrMj8z7bLX"  # Replace with your actual token

# 4: Run Your App (With sharing - requires ngrok token)

from pyngrok import ngrok

import time

import threading



# Set your ngrok authentication token (replace ngrok_token with your actual token)

ngrok.set_auth_token(ngrok_token)



# Function to launch the Streamlit app using a system command

def run_app():

    !streamlit run app.py --server.headless true --server.port 8501



# Terminate any active ngrok tunnels before starting a new one

ngrok.kill()



# Start the Streamlit app in a separate thread so the script can continue running

app_thread = threading.Thread(target=run_app)

app_thread.start()



# Allow time for the Streamlit app to fully start before creating the tunnel

time.sleep(10)



# Create a public URL using ngrok and display it

try:

    public_url = ngrok.connect(8501)

    print("🚀 Your app is live!")

    print(f"🌐 Share this link: {public_url}")

    print("📱 Anyone can access your app with this link!")

except:

    print("⚠️ Need ngrok token for sharing. App is running locally.")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.55.248.60:8501

🚀 Your app is live!
🌐 Share this link: NgrokTunnel: "https://2e9f6fde9867.ngrok-free.app" -> "http://localhost:8501"
📱 Anyone can access your app with this link!


#Disable all the services

In [ ]:

# # --- FINAL CLEANUP SCRIPT FOR COLAB ---

# PROJECT_ID = "i-monolith-468706-i9"
# REGION = "us-central1"
# BUCKET_NAME = "bucket_jonah"

# # 1. Delete all objects in the bucket, then delete the bucket
# !gcloud storage rm -r gs://$BUCKET_NAME --project=$PROJECT_ID || echo "Bucket not found or already deleted"
# !gcloud storage buckets delete gs://$BUCKET_NAME --project=$PROJECT_ID --quiet || echo "Bucket already deleted"

# # 2. Delete all Vertex AI endpoints
# !for eid in $(gcloud ai endpoints list --region=$REGION --project=$PROJECT_ID --format="value(name)"); do \
#     gcloud ai endpoints delete $eid --region=$REGION --project=$PROJECT_ID --quiet; \
# done

# # 3. Delete all Vertex AI models
# !for mid in $(gcloud ai models list --region=$REGION --project=$PROJECT_ID --format="value(name)"); do \
#     gcloud ai models delete $mid --region=$REGION --project=$PROJECT_ID --quiet; \
# done

# # 4. Stop all Vertex AI Workbench notebooks
# !for nid in $(gcloud notebooks instances list --location=$REGION --project=$PROJECT_ID --format="value(name)"); do \
#     gcloud notebooks instances stop $nid --location=$REGION --project=$PROJECT_ID --quiet; \
# done

# # 5. Disable APIs (force disable dependent services)
# !gcloud services disable aiplatform.googleapis.com --project=$PROJECT_ID --quiet || echo "Vertex AI API already disabled"
# !gcloud services disable storage.googleapis.com --project=$PROJECT_ID --quiet --disable-dependent-services || echo "Storage API already disabled"

# # 6. Disable the service account (ignore if not found)
# !gcloud iam service-accounts disable vertex-sa@$PROJECT_ID.iam.gserviceaccount.com --project=$PROJECT_ID || echo "Service account not found"

# print("✅ Final cleanup complete. All costing services stopped for project:", PROJECT_ID)
